In [1]:
import config
from model import Translator
from transformers import AutoTokenizer

src_tokenizer = AutoTokenizer.from_pretrained(config.SRC_MODEL_NAME)
tgt_tokenizer = AutoTokenizer.from_pretrained(config.TGT_MODEL_NAME)

translator = Translator.load_from_checkpoint(
    "../checkpoints/last.ckpt",
    src_tokenizer=src_tokenizer,
    tgt_tokenizer=tgt_tokenizer,
)

/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias'

In [2]:
import torch


def translate(text: str):
    self = translator

    src_token_ids, src_attention_mask = src_tokenizer(
        text, return_token_type_ids=False, return_tensors="pt"
    ).values()

    src_token_ids = src_token_ids.to(self.device)
    src_attention_mask = src_attention_mask.to(self.device)

    print(src_token_ids, src_attention_mask)

    tgt_token_ids = torch.tensor([[tgt_tokenizer.cls_token_id]], device=self.device)
    tgt_attention_mask = torch.tensor([[1]], device=self.device)

    for _ in range(config.MAX_SEQ_LEN):
        logits = translator(
            src_token_ids, tgt_token_ids, src_attention_mask, tgt_attention_mask
        )

        next_tgt_token_id = torch.argmax(logits[:, -1, :], keepdim=True, dim=-1)
        tgt_token_ids = torch.cat([tgt_token_ids, next_tgt_token_id], dim=-1)
        tgt_attention_mask = torch.cat(
            [
                tgt_attention_mask,
                torch.ones_like(next_tgt_token_id, dtype=torch.int64)
                if next_tgt_token_id != tgt_tokenizer.pad_token_id
                else torch.zeros_like(next_tgt_token_id, dtype=torch.int64),
            ],
            dim=-1,
        )

        if next_tgt_token_id == tgt_tokenizer.sep_token_id:
            break

    return tgt_tokenizer.decode(tgt_token_ids[0])


In [5]:
import heapq


class PriorityQueue:
    def __init__(self, key=lambda x: x, mode="min"):
        self.heap = []
        self.mode = mode

        if mode == "max":
            self.key = lambda x: -key(x)
        elif mode == "min":
            self.key = key

    def push(self, item):
        heapq.heappush(self.heap, (self.key(item), item))

    def pop(self):
        return heapq.heappop(self.heap)[1]

    def __len__(self):
        return len(self.heap)

    def clear(self):
        self.heap.clear()

    def empty(self):
        return len(self.heap) == 0

In [13]:
def beam_translate(text: str, max_translation_length: int = 50, beam_size: int = 3):
    self = translator

    src_token_ids, src_attention_mask = src_tokenizer(
        text, return_token_type_ids=False, return_tensors="pt"
    ).values()

    src_token_ids = src_token_ids.to(self.device)
    src_attention_mask = src_attention_mask.to(self.device)

    tgt_token_ids = torch.tensor([[tgt_tokenizer.cls_token_id]], device=self.device)
    tgt_attention_mask = torch.tensor([[1]], device=self.device)

    # =========================== #
    heap = PriorityQueue(key=lambda x: x[0], mode="min")
    heap.push((1.0, tgt_token_ids, tgt_attention_mask, False))

    ret = []
    for _ in range(max_translation_length):
        # Completed
        if len(ret) == beam_size:
            break

        # Keep track of the top k candidates
        while len(heap) > beam_size:
            heap.pop()

        norm_prob = 0
        mem = []

        while not heap.empty():
            # P(T1:Tn-1)
            # tgt_token_ids.shape == (1, seq_len)
            tgt_seq_prob, tgt_token_ids, tgt_attention_mask, completed = heap.pop()

            if completed:
                ret.append((tgt_seq_prob, tgt_tokenizer.decode(tgt_token_ids.squeeze_())))
                continue

            norm_prob += tgt_seq_prob

            logits = translator(
                src_token_ids, tgt_token_ids, src_attention_mask, tgt_attention_mask
            )

            # (vocab_size,)
            token_probs = torch.softmax(logits[:, -1, :], dim=-1).squeeze_()

            # P(Tn | T1:Tn-1)
            top_k_token_probs, top_k_token_ids = torch.topk(
                token_probs, beam_size, largest=True
            )

            for i in range(beam_size):
                next_token_id = top_k_token_ids[i]
                next_token_prob = top_k_token_probs[i]
                completed = next_token_id == tgt_tokenizer.sep_token_id

                mem.append((
                    tgt_seq_prob * next_token_prob,
                    torch.cat((tgt_token_ids, next_token_id.view(1, 1)), dim=-1),
                    torch.cat((tgt_attention_mask, torch.tensor([[1]], device=self.device)), dim=-1),
                    completed
                ))

        for tgt_seq_prob, tgt_token_ids, tgt_attention_mask, completed in mem:
            tgt_seq_prob /= norm_prob # normalize
            heap.push((tgt_seq_prob, tgt_token_ids, tgt_attention_mask, completed))

    while len(ret) < beam_size and not heap.empty():
        tgt_seq_prob, tgt_token_ids, tgt_attention_mask, completed = heap.pop()
        
        decoded_seq = tgt_tokenizer.decode(tgt_token_ids[0])
        ret.append((tgt_seq_prob, decoded_seq))

    return ret

In [18]:
from datasets import load_dataset

iwslt = load_dataset("mt_eng_vietnamese", "iwslt2015-vi-en", split="test")

Found cached dataset mt_eng_vietnamese (/home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)


In [20]:
iwslt[0]["translation"]

{'en': 'When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;',
 'vi': 'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;'}